In [2]:
import torch

In [3]:
B,T,C = 3,5,2
x = torch.zeros(B,T,C)
for b in range(B):
    for t in range(T):
        x[b,t,:] = b*10+t

print(x.shape)
print(x[:,:,0])  # B, T
print(x[0,:,:])  # T, C

torch.Size([3, 5, 2])
tensor([[ 0.,  1.,  2.,  3.,  4.],
        [10., 11., 12., 13., 14.],
        [20., 21., 22., 23., 24.]])
tensor([[0., 0.],
        [1., 1.],
        [2., 2.],
        [3., 3.],
        [4., 4.]])


In [4]:
y = torch.zeros_like(x)
for t in range(T):
    y[:,t,:] = x[:,0:t+1,:].mean(dim=1)
print(y[:,:,0])  # B, T
print(y[0,:,:])  # T, C

tensor([[ 0.0000,  0.5000,  1.0000,  1.5000,  2.0000],
        [10.0000, 10.5000, 11.0000, 11.5000, 12.0000],
        [20.0000, 20.5000, 21.0000, 21.5000, 22.0000]])
tensor([[0.0000, 0.0000],
        [0.5000, 0.5000],
        [1.0000, 1.0000],
        [1.5000, 1.5000],
        [2.0000, 2.0000]])


In [5]:
t = torch.tril(torch.ones((T,T)))
w = t / t.sum(dim=1, keepdim=True)
print(w)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000]])


In [6]:
y2 = w @ x
print(y2[:,:,0])  # B, T
print(y2[0,:,:])  # T, C
print((y==y2).all())

tensor([[ 0.0000,  0.5000,  1.0000,  1.5000,  2.0000],
        [10.0000, 10.5000, 11.0000, 11.5000, 12.0000],
        [20.0000, 20.5000, 21.0000, 21.5000, 22.0000]])
tensor([[0.0000, 0.0000],
        [0.5000, 0.5000],
        [1.0000, 1.0000],
        [1.5000, 1.5000],
        [2.0000, 2.0000]])
tensor(True)


In [9]:
# Method 3
t = torch.tril(torch.ones((T,T)))
w = torch.zeros((T,T))                     # weights, this will comefrom key and query
w = w.masked_fill(t==0, float('-inf'))     # mask future tokens
w = torch.softmax(w, dim=-1)               # normalize to 0..1 sum=1
print(w)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000]])


In [10]:
y3 = w @ x                     # weighted sum of token info
print(y3[:,:,0])  # B, T
print(y3[0,:,:])  # T, C
print((y==y3).all())

tensor([[ 0.0000,  0.5000,  1.0000,  1.5000,  2.0000],
        [10.0000, 10.5000, 11.0000, 11.5000, 12.0000],
        [20.0000, 20.5000, 21.0000, 21.5000, 22.0000]])
tensor([[0.0000, 0.0000],
        [0.5000, 0.5000],
        [1.0000, 1.0000],
        [1.5000, 1.5000],
        [2.0000, 2.0000]])
tensor(True)
